In [4]:
!pip install docplex cplex

In [5]:
#Importing cplex API
import sys
import cplex
import docplex.cp

#Importing numpy and random generator
import numpy as np
rand = np.random

#Importing pyplot
import matplotlib.pyplot as plt

#Importing docplex model
from docplex.mp.model import Model
import itertools

#Imports
from enum import Enum
from cplex.callbacks import UserCutCallback, LazyConstraintCallback

In [ ]:
class LazyCallback(LazyConstraintCallback):
    """Lazy constraint callback to enforce the capacity constraints.

    If used then the callback is invoked for every integer feasible
    solution CPLEX finds. For each location j it checks whether
    constraint

    sum(c in C) supply[c][j] <= (|C| - 1) * used[j]
    sum(e in C) y[e] <= (|C| - 1)

    is satisfied. If not then it adds the violated constraint as lazy
    constraint.
    """

    # Callback constructor. Fields 'locations', 'clients', 'used', 'supply'
    # are set externally after registering the callback.
    def __init__(self, env):
      super().__init__(env)

    def __call__(self):
      

        for j in C:
            served = sum(self.get_values(
                [self.y[e] for e in self.C]))
            if served > (len(self.C) - 1.0):
                print('Adding lazy constraint %s <= %d' %
                      (' + '.join(['y(%d)' % (x) for x in self.C]),
                       len(self.clients) - 1))
                self.add(constraint=cplex.SparsePair(
                    [self.supply[c][j] for c in self.clients] + [self.used[j]],
                    [1.0] * len(self.clients) + [-(len(self.clients) - 1)]),
                    sense='L',
                    rhs=0.0)

In [9]:
class Type(Enum):
  AD = 1
  B = 2
  P = 3


class Vertex:
  def __init__(self, tp, delta):
    self.tp = tp
    # (delta_in, delta_out) -> delta_in = minus / delta_out = plus
    self.delta = delta

In [72]:
E = ["E1", "E2", "E3", "E4", "E5", "E6", "E7", "E8"]  # Edges
#N = [i for i in range(1, 9)]                   # stazioni senza deposito
#E = [i for i in N]
V = []  # Vertex
w = [1, 1, 1, 1, 1, 1, 1, 1]  # Weigths

In [73]:
V.append(Vertex(tp=Type.AD, delta=([],["E1"])))
V.append(Vertex(tp=Type.P, delta=(["E1"],["E2"])))
V.append(Vertex(tp=Type.P, delta=(["E2"],["E3", "E4"])))
V.append(Vertex(tp=Type.P, delta=(["E3"],["E5"])))
V.append(Vertex(tp=Type.P, delta=(["E4", "E6"],["E7"])))
V.append(Vertex(tp=Type.P, delta=(["E7"],["E6", "E8"])))
V.append(Vertex(tp=Type.B, delta=(["E5"],[])))
V.append(Vertex(tp=Type.B, delta=(["E8"],[])))

In [74]:
# Model
model = Model('KEP_model')

# Decision variables
y = model.binary_var_dict(len(E), name='y')
f_in = model.continuous_var_dict(len(V), name='f_in')
f_out = model.continuous_var_dict(len(V), name='f_out')

In [75]:
#Function to minimize
model.maximize(model.sum(w[e] * y[e] for e,_ in enumerate(E)))

In [76]:
#Constraints
model.add_constraints(model.sum(y[e] for e,_ in enumerate(V[v].delta[0])) == f_in[v] for v,_ in enumerate(V))
model.add_constraints(model.sum(y[e] for e,_ in enumerate(V[v].delta[1])) == f_out[v] for v,_ in enumerate(V))
model.add_constraints(f_out[v] <= f_in[v] for v,_ in enumerate(V) if V[v].tp == Type.P)
model.add_constraints(f_in[v] <= 1 for v,_ in enumerate(V) if V[v].tp == Type.P)
model.add_constraints(f_out[v] <= 1 for v,_ in enumerate(V) if (V[v].tp == Type.AD or V[v].tp == Type.B))

[docplex.mp.LinearConstraint[](f_out_0,LE,1),
 docplex.mp.LinearConstraint[](f_out_6,LE,1),
 docplex.mp.LinearConstraint[](f_out_7,LE,1)]

In [77]:
solution = model.solve(log_output=True)
model.solve(log_output=True)

Version identifier: 20.1.0.1 | 2021-12-07 | 9dfdf6686
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 20 rows and 15 columns.
Aggregator did 9 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.03 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.03 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.03 ticks)
Version identifier: 20.1.0.1 | 2021-12-07 | 9dfdf6686
CPXPARAM_Read_DataCheck                          1

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time

docplex.mp.solution.SolveSolution(obj=7,values={y_0:1,y_2:1,y_3:1,y_4:1,..

In [78]:
print(solution)

solution for: KEP_model
objective: 7
y_0=1
y_2=1
y_3=1
y_4=1
y_5=1
y_6=1
y_7=1
f_in_1=1.000
f_in_2=1.000
f_in_3=1.000
f_in_4=1.000
f_in_5=1.000
f_in_6=1.000
f_in_7=1.000
f_out_0=1.000
f_out_1=1.000
f_out_2=1.000
f_out_3=1.000
f_out_4=1.000
f_out_5=1.000

